## LFW Dataset Benchmarking with Mediapipe + Facenet

In [ ]:
import time
import numpy as np
import pandas as pd
from deepface import DeepFace
from sklearn.model_selection import train_test_split
import psutil
import os
import cv2

In [ ]:

def load_lfw_dataset():
    images, labels = [], []
    image_dir = "C:/Dataset_all/lfw"
    for person_dir in os.listdir(image_dir):
        person_path = os.path.join(image_dir, person_dir)
        for img_name in os.listdir(person_path):
            img_path = os.path.join(person_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (160, 160))  # Resize images for consistency
            images.append(img)
            labels.append(person_dir)
    return images, labels

def preprocess_image(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

def get_model_memory_usage(detector, embedder):
    process = psutil.Process(os.getpid())
    initial_mem = process.memory_info().rss
    _ = DeepFace.analyze("C:/Dataset_all/face1.jpg", detector_backend=detector, model_name=embedder)
    final_mem = process.memory_info().rss
    model_size = (final_mem - initial_mem) / (1024 ** 2)
    return model_size


In [ ]:


images, labels = load_lfw_dataset()
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=42)

accuracies, times = [], []
model_size = get_model_memory_usage("mediapipe", "Facenet")

for image, true_label in zip(X_test, y_test):
    image = preprocess_image(image)
    start_time = time.time()
    prediction = DeepFace.analyze(image, detector_backend="mediapipe", model_name="Facenet")
    elapsed_time = time.time() - start_time
    times.append(elapsed_time * 1000)
    predicted_label = prediction['identity']  # Use the actual identity prediction
    accuracies.append(predicted_label == true_label)

avg_accuracy = np.mean(accuracies) * 100
avg_time = np.mean(times)

results = pd.DataFrame({
    "Model Combination": ["Mediapipe + Facenet"],
    "Dataset": ["LFW"],
    "Accuracy": [avg_accuracy],
    "Speed (ms)": [avg_time],
    "Model Size (MB)": [model_size]
})

if os.path.exists("results.csv"):
    results.to_csv("results.csv", mode='a', header=False, index=False)
else:
    results.to_csv("results.csv", index=False)


## CASIA-WebFace Dataset Benchmarking with MTCNN + ArcFace

In [ ]:
import time
import numpy as np
import pandas as pd
from deepface import DeepFace
from sklearn.model_selection import train_test_split
import psutil
import os
import cv2

In [ ]:


def load_casia_dataset():
    images, labels = [], []
    image_dir = "C:/Dataset_all/casia-webface"
    for person_dir in os.listdir(image_dir):
        person_path = os.path.join(image_dir, person_dir)
        for img_name in os.listdir(person_path):
            img_path = os.path.join(person_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (112, 112))  # Resize images for the specific ArcFace input size
            images.append(img)
            labels.append(person_dir)
    return images, labels

def preprocess_image(image):
    return cv2.equalizeHist(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))  # Grayscale and equalize histogram

def get_model_memory_usage(detector, embedder):
    process = psutil.Process(os.getpid())
    initial_mem = process.memory_info().rss
    _ = DeepFace.analyze("C:/Dataset_all/face2.jpg", detector_backend=detector, model_name=embedder)
    final_mem = process.memory_info().rss
    model_size = (final_mem - initial_mem) / (1024 ** 2)
    return model_size

In [ ]:


images, labels = load_casia_dataset()
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=42)

accuracies, times = [], []
model_size = get_model_memory_usage("mtcnn", "ArcFace")

for image, true_label in zip(X_test, y_test):
    image = preprocess_image(image)
    start_time = time.time()
    prediction = DeepFace.analyze(image, detector_backend="mtcnn", model_name="ArcFace")
    elapsed_time = time.time() - start_time
    times.append(elapsed_time * 1000)
    predicted_label = prediction['identity']
    accuracies.append(predicted_label == true_label)

avg_accuracy = np.mean(accuracies) * 100
avg_time = np.mean(times)

results = pd.DataFrame({
    "Model Combination": ["MTCNN + ArcFace"],
    "Dataset": ["CASIA-WebFace"],
    "Accuracy": [avg_accuracy],
    "Speed (ms)": [avg_time],
    "Model Size (MB)": [model_size]
})

if os.path.exists("results.csv"):
    results.to_csv("results.csv", mode='a', header=False, index=False)
else:
    results.to_csv("results.csv", index=False)


## VGGFace2 Dataset Benchmarking with Mediapipe + VGG-Face

In [ ]:
import time
import numpy as np
import pandas as pd
from deepface import DeepFace
from sklearn.model_selection import train_test_split
import psutil
import os
import cv2

In [ ]:


def load_vggface2_dataset():
    images, labels = [], []
    image_dir = "C:/Dataset_all/vggface2"
    for person_dir in os.listdir(image_dir):
        person_path = os.path.join(image_dir, person_dir)
        for img_name in os.listdir(person_path):
            img_path = os.path.join(person_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))  # Resize for VGG-Face
            images.append(img)
            labels.append(person_dir)
    return images, labels

def preprocess_image(image):
    mean = np.array([129.1863, 104.7624, 93.5940])  # VGG-Face mean subtraction values
    return image - mean  # Subtract mean from image

def get_model_memory_usage(detector, embedder):
    process = psutil.Process(os.getpid())
    initial_mem = process.memory_info().rss
    _ = DeepFace.analyze("C:/Dataset_all/face3.jpg", detector_backend=detector, model_name=embedder)
    final_mem = process.memory_info().rss
    model_size = (final_mem - initial_mem) / (1024 ** 2)
    return model_size

In [ ]:


images, labels = load_vggface2_dataset()
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=42)

accuracies, times = [], []
model_size = get_model_memory_usage("mediapipe", "VGG-Face")

for image, true_label in zip(X_test, y_test):
    image = preprocess_image(image)
    start_time = time.time()
    prediction = DeepFace.analyze(image, detector_backend="mediapipe", model_name="VGG-Face")
    elapsed_time = time.time() - start_time
    times.append(elapsed_time * 1000)
    predicted_label = prediction['identity']
    accuracies.append(predicted_label == true_label)

avg_accuracy = np.mean(accuracies) * 100
avg_time = np.mean(times)

results = pd.DataFrame({
    "Model Combination": ["Mediapipe + VGG-Face"],
    "Dataset": ["VGGFace2"],
    "Accuracy": [avg_accuracy],
    "Speed (ms)": [avg_time],
    "Model Size (MB)": [model_size]
})

if os.path.exists("results.csv"):
    results.to_csv("results.csv", mode='a', header=False, index=False)
else:
    results.to_csv("results.csv", index=False)
